In [1]:
!pip install pgmpy

  Obtaining dependency information for pgmpy from https://files.pythonhosted.org/packages/eb/9a/2fcb6fdfd998a016cef29ca3eab30b98b6c232b6e9a0444df07f0ad47f8d/pgmpy-0.1.24-py3-none-any.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/59/1f/4975d1ab3ed2244053876321ef65bc02935daed67da76c6e7d65900772a3/torch-2.2.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ----------------------- -------------- 41.0/65.5 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 65.5/65.5 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------ --------------------------------- 0.3/2.0 MB 9.9 MB/s eta 0:00:

In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
# Define the network structure
network = BayesianNetwork([('Burglary', 'Alarm'),
                           ('Earthquake', 'Alarm'),
                           ('Alarm', 'JohnCalls'),
                           ('Alarm', 'MaryCalls')])

In [4]:
# Define the Conditional Probability Distributions (CPDs)
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.999], [0.001]])
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.998], [0.002]])
cpd_alarm = TabularCPD(variable='Alarm', variable_card=2,
                       values=[[0.999, 0.71, 0.06, 0.05],
                               [0.001, 0.29, 0.94, 0.95]],
                       evidence=['Burglary', 'Earthquake'],
                       evidence_card=[2, 2])
cpd_john_calls = TabularCPD(variable='JohnCalls', variable_card=2,
                             values=[[0.95, 0.1],
                                     [0.05, 0.9]],
                             evidence=['Alarm'],
                             evidence_card=[2])
cpd_mary_calls = TabularCPD(variable='MaryCalls', variable_card=2,
                             values=[[0.99, 0.01],
                                     [0.01, 0.99]],
                             evidence=['Alarm'],
                             evidence_card=[2])

In [5]:
# Add CPDs to the network
network.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john_calls, cpd_mary_calls)

In [6]:
# Initialize the inference engine
inference = VariableElimination(network)

In [7]:
# Perform exact inference
evidence = {'JohnCalls': 1, 'MaryCalls': 0}  # John called (1) and Mary didn't call (0) as evidence
query_variable = 'Burglary'

In [8]:
result = inference.query(variables=[query_variable], evidence=evidence)
print(result)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9998 |
+-------------+-----------------+
| Burglary(1) |          0.0002 |
+-------------+-----------------+
